In [ ]:
import requests
import pandas as pd
import json
import io

# API configuration
API_KEY = #insert your api key after getting it from odds api
SPORT = "basketball_nba"  # Changed from "basketball_ncaab"
REGION = "us"           # "us", "uk", "eu", etc.
MARKETS = "spreads"     # Head-to-head, spreads, totals
URL = f"https://api.the-odds-api.com/v4/sports/{SPORT}/odds?regions={REGION}&markets={MARKETS}&apiKey={API_KEY}"

def lambda_handler(event, context):
    # Fetch data from the API
    response = requests.get(URL)
    if response.status_code != 200:
        print(f"Error fetching data: {response.status_code}, {response.text}")
        return {
            'statusCode': response.status_code,
            'body': response.text
        }
    
    data = response.json()
    odds_data = []
    
    # Process each game’s data
    for game in data:
        game_name = f"{game.get('home_team')} vs {game.get('away_team')}"
        for bookmaker in game.get("bookmakers", []):
            for market in bookmaker.get("markets", []):
                if market.get("key") != "spreads":
                    continue
                for outcome in market.get("outcomes", []):
                    decimal_odds = outcome.get("price")
                    odds_entry = {
                        "Game": game_name,
                        "Bookmaker": bookmaker.get("title"),
                        "Market": market.get("key"),
                        "Outcome": outcome.get("name"),
                        "DecimalOdds": decimal_odds,
                        "AmericanOdds": decimal_to_american(decimal_odds),
                        "Point Value": outcome.get("point")
                    }
                    odds_data.append(odds_entry)
    
    # Create DataFrame
    df = pd.DataFrame(odds_data)
    print("Sample Data:")
    print(df.head())
    
    # Save the CSV to S3 (update bucket name accordingly)
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)
    s3 = boto3.client('s3')
    bucket_name = "your-s3-bucket-name"  # Replace with your actual S3 bucket name
    s3.put_object(Bucket=bucket_name, Key="odds_data.csv", Body=csv_buffer.getvalue())
    
    # Find arbitrage opportunities in spread markets:
    # For each game and spread line, pick the best (i.e., highest) decimal odds per outcome (usually one per team)
    spread_df = df[df["Market"] == "spreads"]
    arbitrage_opportunities = []
    
    # Group by Game and the spread (Point Value)
    for (game, point_value), group in spread_df.groupby(["Game", "Point Value"]):
        # Identify the distinct outcomes (teams)
        outcomes = group["Outcome"].unique()
        if len(outcomes) < 2:
            continue  # Skip if we don’t have two sides to bet on
        
        # For each outcome, select the row offering the best (highest) decimal odds
        best_rows = group.sort_values("DecimalOdds", ascending=False).groupby("Outcome").first().reset_index()
        if len(best_rows) != 2:
            continue
        
        # Extract the best odds for the two outcomes
        row1 = best_rows.iloc[0]
        row2 = best_rows.iloc[1]
        odds1 = row1["DecimalOdds"]
        odds2 = row2["DecimalOdds"]
        
        # Calculate implied probabilities for each outcome
        imp_prob1 = 1 / odds1
        imp_prob2 = 1 / odds2
        
        # Check for arbitrage: the sum of implied probabilities is less than 1
        if (imp_prob1 + imp_prob2) < 1:
            profit_margin = (1 - (imp_prob1 + imp_prob2)) * 100
            arbitrage_opportunities.append({
                "Game": game,
                "Spread": point_value,
                "Outcome1": row1["Outcome"],
                "Bookmaker1": row1["Bookmaker"],
                "Odds1": row1["AmericanOdds"],
                "Outcome2": row2["Outcome"],
                "Bookmaker2": row2["Bookmaker"],
                "Odds2": row2["AmericanOdds"],
                "ProfitMargin(%)": round(profit_margin, 2)
            })
            print(f"Arbitrage Opportunity in {game} with spread {point_value}:")
            print(f"  Bet on {row1['Outcome']} at {row1['Bookmaker']} (Odds: {row1['AmericanOdds']})")
            print(f"  Bet on {row2['Outcome']} at {row2['Bookmaker']} (Odds: {row2['AmericanOdds']})")
            print(f"  Profit Margin: {round(profit_margin,2)}%\n")
    
    return {
        'statusCode': 200,
        'body': json.dumps({'arbitrage_opportunities': arbitrage_opportunities})
    }

def decimal_to_american(decimal_odds):
    """Convert decimal odds to American odds format."""
    if decimal_odds >= 2.00:
        return round((decimal_odds - 1) * 100)
    else:
        return round(-100 / (decimal_odds - 1))
